In [1]:
# switch to the project directory
%cd ..
# working directory should be ../FSE

C:\Users\admin\Desktop\research\FSE


In [2]:
from pdi.constants import (
    NUM_WORKERS,
)

from pdi.data.constants import N_COLUMNS
from pdi.data.types import Split


In [3]:
sweep_configuration = {
    "method": "random",
    "metric": {"goal": "maximize", "name": "val_f1"},
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 10,
        "eta": 2,
    },
    "parameters": {
        "bs": {"value": 256},
        "max_epochs": {"value": 100},
        "start_lr": {"value": 5e-3},
        "patience": {"value": 10},
        "patience_threshold": {"value": 0.001},
        "h0": {"values": [16, 32, 64, 128, 256]},
        "h_layers": {"values": [1, 2, 3]},
        "h_scale": {"values": [0.125, 0.25, 0.5, 0.75]},
        "dropout": {"min": 0.0, "max": 0.4},
        "gamma": {"value": 0.9}
    },
}

In [4]:
import torch
import torch.nn as nn

torch.cuda.set_device(0)
device = torch.device("cuda")


In [5]:
from pdi.models.utils import NeuralNet
from pdi.data.preparation import DeletePreparation


data_preparation = DeletePreparation()
def create_model():
    hidden_layers = [wandb.config.h0]
    for i in range(wandb.config.h_layers - 1):
        hidden_layers.append(int(wandb.config.h_scale * hidden_layers[-1]))
    
    return NeuralNet(
        [N_COLUMNS, *hidden_layers, 1],
        nn.ReLU,
        wandb.config.dropout,
    ).to(device)



In [6]:
import wandb
from pdi.train import train

experiment_name = "DeleteSweep"

target_code = 321
def main():
    run = wandb.init()
    pos_weight = torch.tensor(data_preparation.pos_weight(target_code)).float().to(device)
    train_loader, val_loader = data_preparation.prepare_dataloaders(wandb.config.bs, NUM_WORKERS, [Split.TRAIN, Split.VAL])
    wandb.log({"pos_weight": pos_weight.item()})
    model = create_model()
    train(model, target_code, device, train_loader, val_loader, pos_weight)

sweep_id = wandb.sweep(sweep=sweep_configuration, project=experiment_name)
wandb.agent(sweep_id, function=main)

        

Create sweep with ID: rw9b4d85
Sweep URL: https://wandb.ai/mkasak/DeleteSweep/sweeps/rw9b4d85


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 3s0rjbmp with config:
wandb: 	bs: 256
wandb: 	dropout: 0.01982799094681855
wandb: 	gamma: 0.9
wandb: 	h0: 32
wandb: 	h_layers: 1
wandb: 	h_scale: 0.5
wandb: 	max_epochs: 100
wandb: 	patience: 10
wandb: 	patience_threshold: 0.001
wandb: 	start_lr: 0.005
wandb: Currently logged in as: mkasak. Use `wandb login --relogin` to force relogin


100%|███████████████████████████████████████████████████████████████████████████████| 600/600 [00:04<00:00, 128.31it/s]


Epoch: 0, F1: 0.4287


100%|███████████████████████████████████████████████████████████████████████████████| 600/600 [00:04<00:00, 128.59it/s]


...

100%|███████████████████████████████████████████████████████████████████████████████| 600/600 [00:05<00:00, 116.01it/s]


Epoch: 43, F1: 0.7421


  0%|                                                                                         | 0/2194 [00:05<?, ?it/s]


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▄▂▄▄▄▅▄▃▂▃▃▃▂▂▁▂▄▄▂▂▂▁▁▃▂▂▂▂▃▃▁▁▁▃▁▂▂▂
pos_weight,▁
scheduled_lr,█▇▇▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_f1,▁▃▄▂▁▃▄▃▆▆▇▇▇▆▇▇▇▇▇▇▇▇███▇▇███████████▇█
val_loss,█▅▆▅▅▄▃▆▃▂▂▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_precision,▁▂▅▁▁▃▃▂▇▆▇▆▆▆▇▇████▇████▇▇███▇████████▇
val_recall,▇▅▁██▆▆▆▄▅▄▆▆▆▅▆▅▅▅▄▅▅▆▅▆▅▆▅▅▆▆▆▆▆▆▆▆▆▅▆
val_threshold,▁▆▆▇▇▆▇████▇█▇██████████████████████████
epoch,43
loss,0.26665


Run uys18mzs errored: RuntimeError('DataLoader worker (pid(s) 1944, 4168, 10228) exited unexpectedly')
wandb: ERROR Run uys18mzs errored: RuntimeError('DataLoader worker (pid(s) 1944, 4168, 10228) exited unexpectedly')
wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
